In [1]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "io.delta:delta-core_2.12:1.0.0",
        "spark.sql.extensions":"io.delta.sql.DeltaSparkSessionExtension",
        "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.delta.catalog.DeltaCatalog"
    }
}

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *  

spark = SparkSession.builder.appName("Lakehouse") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.databricks.delta.schema.autoMerge.enabled","true") \
    .config("spark.databricks.delta.autoOptimize.optimizeWrite","true") \
    .config("spark.databricks.delta.optimizeWrite.enabled","true") \
    .config("spark.databricks.delta.vacuum.parallelDelete.enabled","true") \
    .getOrCreate()

from delta.tables import *
from pyspark.sql.functions import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Get the schema from file to load table

In [49]:
bucket='s3://star-wars-etl-dev'
prefix='raw_data'
table = ['films', 'planets', 'species', 'starships', 'vehicles', 'people']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
spark.sql('select current_date()').show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|current_date()|
+--------------+
|    2023-11-11|
+--------------+

In [21]:
for i in table:
    df = spark.read.json(f"{bucket}/{prefix}/{i}", multiLine=True)
    print(f"table {i}\n {df.schema.json()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

table films
 {"fields":[{"metadata":{},"name":"count","nullable":true,"type":"long"},{"metadata":{},"name":"next","nullable":true,"type":"string"},{"metadata":{},"name":"previous","nullable":true,"type":"string"},{"metadata":{},"name":"results","nullable":true,"type":{"containsNull":true,"elementType":{"fields":[{"metadata":{},"name":"characters","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"created","nullable":true,"type":"string"},{"metadata":{},"name":"director","nullable":true,"type":"string"},{"metadata":{},"name":"edited","nullable":true,"type":"string"},{"metadata":{},"name":"episode_id","nullable":true,"type":"long"},{"metadata":{},"name":"opening_crawl","nullable":true,"type":"string"},{"metadata":{},"name":"planets","nullable":true,"type":{"containsNull":true,"elementType":"string","type":"array"}},{"metadata":{},"name":"producer","nullable":true,"type":"string"},{"metadata":{},"name":"release_date","nullable":true,

In [51]:
%%sql
select current_date()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### load dataframe to bucket

In [97]:
# load json
print(bucket)
print(prefix)
print(table[1])
nam_schema=f'/home/glue_user/workspace/jupyter_workspace/bronze_data/schemas/{table[1]}.json'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://star-wars-etl-dev
raw_data
planets

In [98]:
%%sh
pwd

/home/glue_user/workspace/jupyter_workspace/bronze_data


In [125]:
import json

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
# schema
with open(nam_schema, 'r') as f:
    data = json.load(f)
schema = StructType.fromJson(data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
from datetime import datetime
dat_load=datetime.now()
dat_load=dat_load.strftime('%Y-%m-%d %H:%M:%S')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
df=spark.read.option("multiline","true").json(f"{bucket}/{prefix}/{table[1]}", schema=schema)
df=df.select(explode(col('results')).alias('results'))
df_exploded = df.select("results.*")
df_exploded=df_exploded.withColumn('dat_load',lit(dat_load))
df_exploded = df_exploded.withColumn("dat_load", to_timestamp("dat_load", "yyyy-MM-dd HH:mm:ss"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
df_exploded.createOrReplaceTempView(f'{table[1]}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
df_exploded.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- climate: string (nullable = true)
 |-- created: timestamp (nullable = true)
 |-- diameter: integer (nullable = true)
 |-- edited: timestamp (nullable = true)
 |-- films: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- gravity: string (nullable = true)
 |-- name: string (nullable = true)
 |-- orbital_period: string (nullable = true)
 |-- population: string (nullable = true)
 |-- residents: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rotation_period: string (nullable = true)
 |-- surface_water: string (nullable = true)
 |-- terrain: string (nullable = true)
 |-- url: string (nullable = true)
 |-- dat_load: timestamp (nullable = true)

In [131]:
df_exploded.select('diameter').show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|diameter|
+--------+
+--------+

In [ ]:
bucket='s3://star-wars-etl-dev'
prefix='raw_data'
table = ['films', 'planets', 'species', 'starships', 'vehicles', 'people']

In [12]:
%%sql
select * from films where episode_id=1 limit 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [13]:
df_exploded.write.mode("overwrite").format("delta").save(f'{bucket}/bronze_data/{table[0]}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_exploded.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- characters: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- created: timestamp (nullable = true)
 |-- director: string (nullable = true)
 |-- edited: timestamp (nullable = true)
 |-- episode_id: long (nullable = true)
 |-- opening_crawl: string (nullable = true)
 |-- planets: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- producer: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- species: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- starships: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- vehicles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- dat_load: timestamp (nullable = true)

In [154]:
## check prefix
import boto3
def check_table(bucket, prefix):
    s3 = boto3.client('s3', region_name='us-east-1')
    try:
        response=s3.list_objects(Bucket=bucket, Prefix=prefix, MaxKeys=1)
        if response.get('Contents')[0].get('Size') > 0:
            return True
        return response
    except Exception as e:
        return False

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
value=check_table('star-wars-etl-dev',f'bronze_data/films/')
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
print(value)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [188]:
table = DeltaTable.forPath(spark, f'{bucket}/bronze_data/{table[0]}')

table.alias("persisteddata").merge(df_exploded.alias("newdata"),"persisteddata.episode_id = newdata.episode_id") \
.whenMatchedUpdateAll() \
.whenNotMatchedInsertAll() \
.execute()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DeltaTable' object is not subscriptable
Traceback (most recent call last):
TypeError: 'DeltaTable' object is not subscriptable



In [5]:
df1 = spark.read.format("delta").load(f'{bucket}/bronze_data/{table[0]}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df1.createOrReplaceTempView('films')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- results: struct (nullable = true)
 |    |-- characters: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- created: timestamp (nullable = true)
 |    |-- director: string (nullable = true)
 |    |-- edited: timestamp (nullable = true)
 |    |-- episode_id: long (nullable = true)
 |    |-- opening_crawl: string (nullable = true)
 |    |-- planets: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- producer: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-- species: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- starships: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- title: string (nullable = true)
 |    |-- url: string (nullable = true)
 |    |-- vehicles: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- characters: array (nullable = true)
 |    |-- element: s

In [16]:
%%sql
select 

    episode_id, 
    producer, 
    title, 
    release_date, 
    dat_load 
    
    from films  order by created asc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### verificando a chave

In [4]:
tables = ['films', 'planets', 'species','starships', 'vehicles', 'people']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
my_dict =[ 
    {"nam_table":"films", "id_table":"episode_id"},
    {"nam_table":"planets", "id_table":"name"},
    {"nam_table":"species", "id_table":"name"},
    {"nam_table":"starships", "id_table":"name"},
    {"nam_table":"vehicles", "id_table":"name"},
    {"nam_table":"people", "id_table":"name"},
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
for i in my_dict:
    print(i.get('nam_table'),i.get('id_table'))
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

films episode_id
planets name
species name
starships name
vehicles name
people name

### Verificando merge 

In [157]:
df1 = spark.read.format("delta").load(f's3://star-wars-etl-dev/bronze_data/planets')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
df1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- climate: string (nullable = true)
 |-- created: string (nullable = true)
 |-- diameter: string (nullable = true)
 |-- edited: string (nullable = true)
 |-- films: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- gravity: string (nullable = true)
 |-- name: string (nullable = true)
 |-- orbital_period: string (nullable = true)
 |-- population: string (nullable = true)
 |-- residents: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rotation_period: string (nullable = true)
 |-- surface_water: string (nullable = true)
 |-- terrain: string (nullable = true)
 |-- url: string (nullable = true)
 |-- dat_load: timestamp (nullable = true)

In [161]:
df2=df1.select('name','films','url')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [162]:
df2.show(10, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+----------------------------------------------------------------+---------------------------------+
|name          |films                                                           |url                              |
+--------------+----------------------------------------------------------------+---------------------------------+
|Alderaan      |[https://swapi.dev/api/films/1/, https://swapi.dev/api/films/6/]|https://swapi.dev/api/planets/2/ |
|Aleen Minor   |[]                                                              |https://swapi.dev/api/planets/38/|
|Bespin        |[https://swapi.dev/api/films/2/]                                |https://swapi.dev/api/planets/6/ |
|Bestine IV    |[]                                                              |https://swapi.dev/api/planets/26/|
|Cato Neimoidia|[https://swapi.dev/api/films/6/]                                |https://swapi.dev/api/planets/18/|
|Cerea         |[]                                                      

In [165]:
df2.select('name').drop_duplicates().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

60

In [166]:
df2.select('name').count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

60